In [1]:
import xml.etree.cElementTree as ET
from collections import defaultdict
import re
import pprint

OSM_FILE = "C:\Chaitanya\Jupyter\Data_Wrangling\Project_DataWrangling\houston_texas.osm"
SAMPLE_FILE = "C:\Chaitanya\Jupyter\Data_Wrangling\Project_DataWrangling\houston_texas_sample.osm"

In [2]:
def is_House_Number(elem):
    return (elem.attrib['k'] == "addr:housenumber")

In [ ]:
house_number_re = re.compile(r'^\d+(-?\d)*$')

def audit_house_number(house_numbers, number):
    m = house_number_re.search(number)
    if not m:
        house_numbers.append(number)
    
    return house_numbers



def audit(osmfile):
    osm_file = open(osmfile, "r")
    house_numbers = []
    for event, elem in ET.iterparse(osm_file, events=("start",)):

        if elem.tag == "node" or elem.tag == "way":
            for tag in elem.iter("tag"):
                if is_House_Number(tag):
                    audit_house_number(house_numbers, tag.attrib['v'])
    
    osm_file.close()
    return house_numbers


def update_house_number(house_number):
    street_re = re.compile(r'[a-z]{3}')
    m1 = street_re.search(house_number)
    better_number = house_number
    if m1:
        #print m1.group(), "    ", house_number
        split = house_number.split(" ")
        if (split[1] != "Suite"):
            return split[0]
        else:
            better_number = house_number
    if ('Ste' in better_number):
        better_number = better_number.replace('Ste', 'Suite')

    return better_number
    


def test():
    house_nos = audit(OSM_FILE)
    for house_number in house_nos:
        better_house_number = update_house_number(house_number)
        print house_number, "=>", better_house_number


if __name__ == '__main__':
    test()